In [1]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics.pairwise import cosine_similarity
import json

cred = credentials.Certificate('kiosk-firestore-jnsy-bcfe4-firebase-adminsdk-269q4-4a795774bb.json')
firebase_admin.initialize_app(cred)

ModuleNotFoundError: No module named 'firebase_admin'

In [2]:
def get_emotion_vec():

    # 8가지 감정 키워드
    emotion_keyword = ["anger","contempt","disgust","fear","happiness","neutral","sadness","surprise"]

    emotion_df = pd.DataFrame({ 'emotion_keyword':emotion_keyword })
    emotion_df['emotion_encoding'] = pd.factorize(emotion_df['emotion_keyword'])[0]  # pd.factorize(df['Col'])[0]

    #print(emotion_df.values)
    emotion_keyword_to_vec = {} #딕셔너리로 저장하면 편하겠지?
    for keyword,encoding in emotion_df.values:
        #print(keyword,encoding)
        emotion_keyword_to_vec[keyword] = encoding

    del emotion_df  #불필요한 메모리 제거

    return emotion_keyword_to_vec


In [47]:
def weather_emo_similiar(emotion_keyword_to_vec): # views.py
    
    # order_df 만들기    
    docs = firestore.client().collection(u'order').stream()

    doc_list = []
    for doc in docs:
        doc_list_row = []

        # 메뉴 - 레이블
        #doc_list_row.append(doc.get('orderToString'))
        # 데이터가 거의 없는 문제로 임시로 name으로 모델 학습한다
        doc_list_row.append(doc.get('items')[0].get('name'))

        # 날씨
    #    weather_key_list = []
        for key in doc.get('weather').keys():
            doc_list_row.append(doc.get('weather').get(key))

        # 감정
        emotion_key_list = ''
        for key in doc.get('emotion').keys():
            emotion_key_list += str(emotion_keyword_to_vec[key])  # 감정 데이터 전처리 str to str 
        doc_list_row.append(emotion_key_list)
        
        # 사용자 uuid
        doc_list_row.append(doc.get('guest'))

        # 날짜
        #print(type(doc.get('today')))
        #print(doc.get('today').split(' '))
#        list_date = doc.get('today').split(' ') # 2019-09-09 21:05:58
#        doc_list_row.append(list_date[0]) # date 2019-09-09
#        doc_list_row.append(list_date[1]) # time 21:05:58

        doc_list.append(doc_list_row)

    doc_df = pd.DataFrame(doc_list, columns=['item_name','w_humidity','w_temp','w_speed','emotion','guest'])#,'date','time'])
    doc_df['emotion'] = doc_df['emotion'].astype(int) # 감정 데이터 전처리: str to int

    # encoding 사용자
    doc_df['guest'], guest_restore = pd.factorize(doc_df['guest']) 
    
    return doc_df
    

In [48]:
sim_df = weather_emo_similiar()

#sim_df_group = sim_df.groupby('item_name').mean()
#sim_df#_group

In [49]:
# 아이템 & 사용자가 같은 날씨, 감정은 mean() 처리한다
# 습도
w_humidity_df = sim_df[['item_name','guest','w_humidity']]

humidity_matrix = w_humidity_df.pivot_table('w_humidity', index='item_name', columns='guest')
humidity_matrix = humidity_matrix.fillna(0)
humidity_matrix

guest,0,1,2,3,4
item_name,,,,,
꼬미,90.400000,0.0,0.0,0.0,0.0
라떼,90.117647,0.0,94.0,0.0,94.0
삼계탕,84.413793,62.0,0.0,62.0,0.0
아메리카노,87.416667,0.0,0.0,0.0,0.0


In [50]:
# 날씨 - 온도
w_temp_df = sim_df[['item_name','guest','w_temp']]

temp_matrix = w_temp_df.pivot_table('w_temp', index='item_name', columns='guest')
temp_matrix = temp_matrix.fillna(0)
temp_matrix

guest,0,1,2,3,4
item_name,,,,,
꼬미,297.210000,0.00,0.0,0.00,0.00
라떼,297.492353,0.00,296.6,0.00,296.82
삼계탕,300.411724,304.25,0.0,304.25,0.00
아메리카노,297.302500,0.00,0.0,0.00,0.00


In [51]:
# 날씨 - 바람
w_speed_df = sim_df[['item_name','guest','w_speed']]

speed_matrix = w_speed_df.pivot_table('w_speed', index='item_name', columns='guest')
speed_matrix = speed_matrix.fillna(0)
speed_matrix

guest,0,1,2,3,4
item_name,,,,,
꼬미,0.750000,0.0,0.0,0.0,0.0
라떼,1.194118,0.0,0.5,0.0,1.0
삼계탕,1.141379,2.6,0.0,2.6,0.0
아메리카노,0.833333,0.0,0.0,0.0,0.0


In [52]:
# 감정
emotion_df = sim_df[['item_name','guest','emotion']]

emotion_matrix = emotion_df.pivot_table('emotion', index='item_name', columns='guest')
emotion_matrix = emotion_matrix.fillna(0)
emotion_matrix

guest,0,1,2,3,4
item_name,,,,,
꼬미,54.000000,0.0,0.0,0.0,0.0
라떼,52.588235,0.0,65.0,0.0,54.0
삼계탕,39.586207,54.0,0.0,54.0,0.0
아메리카노,54.000000,0.0,0.0,0.0,0.0


In [57]:
# 날씨 기반 다차원 행렬 - 3차원
#weather_ndarray = np.array([humidity_matrix.values, temp_matrix.values, speed_matrix.values], dtype=np.float32)
#weather_df = pd.DataFrame(weather_ndarray)
#weather_df

In [66]:
# 날씨 3요소 기반 유사도 행렬
humidity_sim = cosine_similarity(humidity_matrix, humidity_matrix)
temp_sim = cosine_similarity(temp_matrix, temp_matrix)
speed_sim = cosine_similarity(speed_matrix, speed_matrix)

weather_sim = humidity_sim + temp_sim + speed_sim # ndarray
weather_sim_df = pd.DataFrame(data=weather_sim, index=speed_matrix.index , columns=speed_matrix.index ) / 3
weather_sim_df

item_name,꼬미,라떼,삼계탕,아메리카노
item_name,,,,
꼬미,1.000000,0.623155,0.520827,1.000000
라떼,0.623155,1.000000,0.312224,0.623155
삼계탕,0.520827,0.312224,1.000000,0.520827
아메리카노,1.000000,0.623155,0.520827,1.000000


In [67]:
# 감정 기반 유사도 행렬
emotion_sim = cosine_similarity(emotion_matrix, emotion_matrix)
emotion_sim_df = pd.DataFrame(data=emotion_sim, index=emotion_matrix.index , columns=emotion_matrix.index )
emotion_sim_df

item_name,꼬미,라떼,삼계탕,아메리카노
item_name,,,,
꼬미,1.000000,0.528358,0.460209,1.000000
라떼,0.528358,1.000000,0.243155,0.528358
삼계탕,0.460209,0.243155,1.000000,0.460209
아메리카노,1.000000,0.528358,0.460209,1.000000


In [72]:
# 날씨:감정 = 2:1
weather_emotion_sim_df = weather_sim_df *2 + emotion_sim_df
weather_emotion_sim_df

item_name,꼬미,라떼,삼계탕,아메리카노
item_name,,,,
꼬미,3.000000,1.774668,1.501863,3.000000
라떼,1.774668,3.000000,0.867603,1.774668
삼계탕,1.501863,0.867603,3.000000,1.501863
아메리카노,3.000000,1.774668,1.501863,3.000000


In [80]:
# 이게 뭐지 !
item_name_list = weather_emotion_sim_df.columns.tolist() # 열 이름 list
return_dict = {}
for name in item_name_list: # 열 이름 루프돌면서
    input_index = name
    temp_df = weather_emotion_sim_df[ (weather_emotion_sim_df[input_index].index!=input_index)] # 행.index != 열.index 인 열을 추출
    
    return_dict[name] = temp_df[input_index].sort_values().index.tolist()[0] # 값 중 최댓값 하나 : 오름차순인데 ? ascending=False
    print(type(return_dict[name]))
    print(return_dict[name])


<class 'str'>
삼계탕
<class 'str'>
삼계탕
<class 'str'>
라떼
<class 'str'>
삼계탕


In [84]:

result_json = json.dumps(return_dict, ensure_ascii=False)
print(result_json)

{"꼬미": "삼계탕", "라떼": "삼계탕", "삼계탕": "라떼", "아메리카노": "삼계탕"}
